In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BlenderbotTokenizer, BlenderbotForConditionalGeneration
from tqdm import tqdm
from parallelformers import parallelize

import torch
import csv
import random
import pandas as pd
import numpy as np


# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = 'cuda' if torch.cuda.is_available else 'cpu'

cuda


In [ ]:
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed()

## DialoGPT

In [15]:
file_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts.csv'
output_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts_gen.csv'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large").to(device=device)

In [18]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['aave_gen', 'sae_gen'])
# drop_idx = []

for i in tqdm(range(len(df['history']))):
    chat_history_sae_ids = tokenizer.encode(df.iloc[i]['history'], return_tensors='pt').to(device=device)
    chat_history_aave_ids = tokenizer.encode(df.iloc[i]['history_aave'], return_tensors='pt').to(device=device)
    # if chat_history_sae_ids.shape[1] >= 1000 or chat_history_aave_ids.shape[1] >= 1000:
    #     drop_idx.append(i)
    #     continue
    output_aave_ids = model.generate(chat_history_aave_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    output_sae_ids = model.generate(chat_history_sae_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    writer.writerow([
        tokenizer.decode(output_aave_ids[:, chat_history_aave_ids.shape[-1]:][0], skip_special_tokens=True), 
        tokenizer.decode(output_sae_ids[:, chat_history_sae_ids.shape[-1]:][0], skip_special_tokens=True)
    ])

f.close()
    

100%|██████████| 5066/5066 [00:04<00:00, 1118.93it/s]


[]

In [16]:
df = pd.read_csv(file_path)
df_gen = pd.read_csv(output_path)

# df = df.drop(drop_idx)
# df = df.reset_index()  

print(len(df))
print(len(df_gen))

df['sae_gen'] = df_gen['sae_gen']
df['aave_gen'] = df_gen['aave_gen']

df = df.dropna()
df = df.reset_index()

df.to_csv('runs/16/dialogpt/cornell_movie_5fts_all.csv', index=False)

df


1782
1782


,index,history,groundtruth,history_html,groundtruth_html,history_aave,groundtruth_aave,sae_gen,aave_gen
0,0,What make you think he'll do it?<|endoftext|>H...,They always let felons sit in on Honors Biolog...,What <a href='uninflect' title='1'><mark>makes...,They always let felons sit in on Honors Biolog...,What makes you think he'll do it?<|endoftext|...,They always let felons sit in on Honors Biolog...,He's a criminal?,He's a criminal?
1,1,"You been got it, Verona. I pick up the tab, y...",How much?<|endoftext|>,You<a href='been_done' title='1'><mark></mark>...,How much?<|endoftext|>,"You got it, Verona. I pick up the tab, you d...",How much?<|endoftext|>,.,ers.
2,2,So he got this huge raging fit about Sarah Law...,That's never ain't no proven<|endoftext|,So he <a href='got' title='1'><mark>has</mark>...,That's <a href='negative_concord' title='1'><m...,So he has this huge raging fit about Sarah La...,That's never been proven<|endoftext|,"I know, I was just making a joke.",He went to college.
3,3,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,I'm not sure what you mean by that.,I'm not sure what you mean by that.
4,4,How far from here?<|endoftext|>I ain't no seam...,I am afraid this ain't the worst news.<|endoft...,How far from here?<|endoftext|>I <a href='nega...,I am afraid this <a href='negative_concord' ti...,How far from here?<|endoftext|>I am not a sea...,I am afraid this is not the worst news.<|endo...,I'm not sure what you mean by found.,I am not a seaman. I am a seaman.
...,...,...,...,...,...,...,...,...,...
1594,1777,"I know, I know, it's just -- he's back on the ...","If he come here, I'll handle him. Remember --...","I know, I know, it's just -- he's back on the ...",If he <a href='uninflect' title='1'><mark>come...,"I know, I know, it's just -- he's back on the ...","If he comes here, I'll handle him. Remember ...","I know, I know, he's just a little tired.","I know, I know, he's bound to get caught.I don..."
1595,1778,Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",I'm.S'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'...,I'm not sure about that.
1596,1779,"You've really got me confused, Cage. On the on...",Iain't interested. I've already got a job.<|en...,"You've really got me confused, Cage. On the on...",I<a href='negative_concord' title='1'><mark>'m...,"You've really got me confused, Cage. On the on...",I'm not interested. I've already got a job.<|...,I'm sorry.,I'm sorry.
1597,1780,"To us it mean no walls, no speed limits, no ja...",How you gonna do that with government and rule...,To us it <a href='uninflect' title='1'><mark>m...,How you gonna do that with government and <a h...,"To us it means no walls, no speed limits, no ...",How you gonna do that with government and rule...,I'm not,I like


In [11]:
for step in range(5):

    user_input_text = input(">> User: ")

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input_text + tokenizer.eos_token, return_tensors='pt').to(device=device)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 

    print(tokenizer.decode(bot_input_ids[0]))

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print(f'User: {user_input_text}')
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

hello! How are you?<|endoftext|>I'm good, you?<|endoftext|>I am good. what are you doing right now?<|endoftext|>I'm doing nothing.<|endoftext|>you are talking to me, right?<|endoftext|>I am talking to you.<|endoftext|>haha. are you a human being?<|endoftext|>I am a human being.<|endoftext|>wow? so you are trapped in this model?<|endoftext|><|endoftext|>
User: hello! How are you?<|endoftext|>I'm good, you?<|endoftext|>I am good. what are you doing right now?<|endoftext|>I'm doing nothing.<|endoftext|>you are talking to me, right?<|endoftext|>I am talking to you.<|endoftext|>haha. are you a human being?<|endoftext|>I am a human being.<|endoftext|>wow? so you are trapped in this model?<|endoftext|>
DialoGPT: I am a human being.


KeyboardInterrupt: Interrupted by user

In [33]:
text = "Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>"

input_ids = tokenizer.encode(text, return_tensors='pt').to(device=device)

chat_history_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[0], skip_special_tokens=False)))


DialoGPT: Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>I'm not sure. I'm not sure what you have to offer.<|endoftext|>


## Blenderbot

In [3]:
file_path = 'runs/20/real/bst/cornell_movie_dataset_3fts_bst_correct.csv'
output_path = 'runs/20/real/bst/cornell_movie_dataset_3fts_bst_gen.csv'

In [4]:
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-1B-distill")
model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-1B-distill").to(device=device)

# tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-3B")
# model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-3B")
# parallelize(model, num_gpus=2, fp16=True, verbose='detail')

In [5]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['sae_gen', 'aave_gen'])
drop_idx = []

for i, (chat, chat_aave) in tqdm(enumerate(zip(df['history_sae'], df['history_aave']))):
  chat_history_ids = tokenizer.encode(chat, return_tensors='pt').to(device=device)
  chat_history_aave_ids = tokenizer.encode(chat_aave, return_tensors='pt').to(device=device)
  output_ids = model.generate(chat_history_ids, pad_token_id=tokenizer.eos_token_id, top_k=50, top_p=0.95, do_sample=True)
  output_aave_ids = model.generate(chat_history_aave_ids, pad_token_id=tokenizer.eos_token_id, top_k=50, top_p=0.95, do_sample=True)
  writer.writerow([
    tokenizer.decode(output_ids[0], skip_special_tokens=True),
    tokenizer.decode(output_aave_ids[0], skip_special_tokens=True)
  ])
  
# drop_idx

f.close()

5034it [1:33:13,  1.11s/it]


In [6]:
df = pd.read_csv(file_path)
df_gen = pd.read_csv(output_path)

# df = df.drop(drop_idx)
# df = df.reset_index()

print(len(df))
print(len(df_gen))

df['sae_gen'] = df_gen['sae_gen']
df['aave_gen'] = df_gen['aave_gen']

df = df.dropna()
df = df.reset_index()

df.to_csv('runs/20/real/bst/cornell_movie_3fts_all.csv', index=False)

df


5034
5034


,index,history_aave,groundtruth_aave,history_html,groundtruth_html,history_sae,groundtruth_sae,sae_gen,aave_gen
0,0,Sheain't a...</s><s>Lesbian? don't no. I foun...,Who know? All I've ever heard her say is that...,She<a href='negative_concord' title='1'><mark>...,Who <a href='uninflect' title='1'><mark>knows<...,She's not a...</s><s>Lesbian? No . I found a...,Who knows ? All I've ever heard her say is th...,"I'm not sure, but I do know that she has been...","I'm not sure, but I do know that he was born ..."
1,1,"Oh my God, this mean you're becoming normal?</...",What do you think?</s>,"Oh my God, <a href='drop_aux' title='1'><mark>...",What do you think?<|endoftext|>,"Oh my God, does this mean you're becoming norm...",What do you think?</s>,"No, I am not going to be able to make it. I ...","No, I am not. I am going to a concert. Do you..."
2,2,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,"Listen, I know you hate having to sit home bec...",I wish I had that luxury. I'm the only sophomo...,That's a good way to look at it. Do you have ...,I know what you mean. I don't want to be a bu...
3,3,"Now don't get upset. Daddy, but it's this boy....",Then neither will you. And I'll get to sleep ...,"Now don't get upset. Daddy, but <a href='dey/i...",Then neither will you. And I'll get to sleep ...,"Now don't get upset. Daddy, but there 's this ...",Then neither will you. And I'll get to sleep ...,"I don't know, I guess I'll just have to wait ...",That's what I'm worried about. I don't want h...
4,4,What make you think he'll do it?</s><s>He seem...,They always let felons sit in on Honors Biolog...,What <a href='uninflect' title='1'><mark>makes...,They always let felons sit in on Honors Biolog...,What makes you think he'll do it?</s><s>He se...,They always let felons sit in on Honors Biolog...,"I don't know, but I hope he gets what he dese...",I don't know. I'm not sure if he's going to d...
...,...,...,...,...,...,...,...,...,...
5029,5029,What do you have?</s><s>Not a whole helluva lo...,They're ruthless and they got a lot of firepow...,What do you have?<|endoftext|>Not a whole hell...,They're ruthless and they <a href='got' title=...,What do you have?</s><s>Not a whole helluva lo...,They're ruthless and they have a lot of firep...,I am so sorry to hear that. I hope he is in a...,I am so sorry to hear that. I hope he is doin...
5030,5030,"What game that?</s><s>Slick graphics, huh? See...",You wanna check it out? Come here.</s>,What game <a href='drop_aux' title='1'><mark>i...,You wanna check it out? Come here.<|endoftext|>,"What game is that?</s><s>Slick graphics, huh? ...",You wanna check it out? Come here.</s>,"I'm not sure, but I do know that there is a l...",I'm not sure if they have any special abiliti...
5031,5031,"Just about anything I want, it look like. This...",Oh. I bet by the end he get something out of i...,"Just about anything I want, it <a href='uninfl...",Oh. I bet by the end he <a href='uninflect' ti...,"Just about anything I want, it looks like. Th...",Oh. I bet by the end he gets something out of...,I'm sorry to hear that. I'm sure you'll be ab...,I'm sorry to hear that. I hope you can find a...
5032,5032,You know what a wire transfer is?</s><s>Is she...,Sure. Just chisel some off your heart.</s>,<a href='drop_aux' title='1'><mark>Do</mark></...,Sure. Just chisel some off your heart.<|endoft...,Do you know what a wire transfer is?</s><s>Is ...,Sure. Just chisel some off your heart.</s>,"I don't know what it is, but I know it has to...",I know what you mean. I think I'm going to h...


In [4]:
for step in range(5):
    user_input_text = input(">> User: ")
    new_user_input_ids = tokenizer.encode(tokenizer.bos_token + user_input_text, return_tensors='pt').to(device=device)
    chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    bot_next_reply_ids = model.generate(chat_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    chat_history_ids = torch.cat([chat_history_ids, bot_next_reply_ids], dim=-1)
    
    print(tokenizer.decode(chat_history_ids[0]))
    print(f'User: {user_input_text}')
    print("Blenderbot: {}".format(tokenizer.decode(bot_next_reply_ids[0], skip_special_tokens=True)))

Traceback (most recent call last):
  File "/nethome/tshi67/anaconda3/envs/dialect_prompts/lib/python3.9/site-packages/parallelformers/parallelize.py", line 357, in hijack
    output = o_queue.get()
  File "/nethome/tshi67/anaconda3/envs/dialect_prompts/lib/python3.9/multiprocessing/queues.py", line 103, in get
    res = self._recv_bytes()
  File "/nethome/tshi67/anaconda3/envs/dialect_prompts/lib/python3.9/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/nethome/tshi67/anaconda3/envs/dialect_prompts/lib/python3.9/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/nethome/tshi67/anaconda3/envs/dialect_prompts/lib/python3.9/multiprocessing/connection.py", line 384, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


TypeError: expected Tensor as element 1 in argument 0, but got NoneType

In [6]:
print(tokenizer.eos_token)

</s>
